# TP : Classification des survivants du Titanic

<center><img src=https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/RMS_Titanic_3.jpg/1280px-RMS_Titanic_3.jpg width=50%></center>

[Le TP est à faire de préférence sur Capytale](https://capytale2.ac-paris.fr/web/c/5ee4-1148472). 

On souhaite prédire si un passager du Titanic a survécu ou non à l'accident, en utilisant l'algorithme des plus proches voisins. On pourra s'inspirer de l'[exemple du cours sur la classification des iris](https://cpge-itc.github.io/itc2/4_knn/exemple/knn_iris.html).

Voici les informations sur chaque passager :  
- `Survived` : 0 = Non, 1 = Oui
- `Pclass` : Classe de ticket (1 = 1ère classe, 2 = 2ème, 3 = 3ème)  
- `Sex` : Genre du passager (`male` ou `female`)  
- `Age` : Age du passager (en années)  
- `Fare` : Tarif du ticket (en dollars)

## Chargement des données avec Pandas

Pandas est un module Python qui permet de manipuler des données sous forme de tableau appelé dataframe :

In [1]:
import pandas as pd

df = pd.read_csv('titanic.csv')
df.head() # pour afficher les 5 premières lignes

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


Ainsi `df` est un tableau contenant 891 lignes et 5 colonnes, chaque ligne correspondant à un passager du Titanic. On peut obtenir le nombres de lignes avec `len(df)`.

Chaque ligne est identifiée par un index, ici $0, 1, 2, ...$. On peut accéder à la ligne d'indice $i$ avec `df.loc[i]` :

In [2]:
df.loc[0]

Survived       0
Pclass         3
Sex         male
Age         22.0
Fare        7.25
Name: 0, dtype: object

`df.loc[i]` donne en fait une series, qui peut être vu comme un tableau à une dimension.

On peut récupérer une colonne (également sous forme de series), par exemple `Age`, avec `df["Age"]` :

In [3]:
df["Age"]

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888    28.0
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

On peut combiner ces deux méthodes pour récupérer une valeur précise, par exemple l'âge du 3ème passager :

In [4]:
df.loc[2, "Age"] # ou df.loc[2]["Age"]

26.0

On peut aussi modifier une valeur avec, par exemple, `df.loc[2, "Age"] = ...`.

On peut parcourir les indices d'un dataframe avec `df.index`. Par exemple, pour trouver le passager le plus vieux :

In [5]:
maxi_age = 0
for i in df.index:
    if df.loc[i, "Age"] > maxi_age:
        maxi_age = df.loc[i, "Age"]
maxi_age

80.0

## Statistiques

````{admonition} Question
 Écrire une fonction `moyenne(df, c)` qui renvoie la moyenne des valeurs de la colonne `c` du dataframe `df`. Quelle est l'âge moyen des passagers du Titanic ? Le prix moyen du ticket ?
````

````{admonition} Question
 Écrire une fonction `ecart_type(df, c)` qui renvoie l'écart-type des valeurs de la colonne `c` du dataframe `df`. On rappelle que l'écart-type d'une série de valeurs $x_1, \ldots, x_n$ est donné par : 
$$\sqrt{\frac{1}{n} \sum_{i=1}^n (x_i - \bar{x})^2}$$
où $\bar{x}$ est la moyenne des valeurs $x_1, ..., x_n$.
````

In [8]:
ecart_type(df, "Age")

13.01238827279366

````{admonition} Question
 Afficher le pourcentage de survivants parmi :
- les hommes
- les femmes
- les passagers de 1ère classe
- les passagers de 2ème classe
- les passagers de 3ème classe
````

## Variables catégorielles

Nous souhaitons modéliser chaque passager par un vecteur de $\mathbb{R}^4$ (car il y a $4$ informations pour chaque passager : âge, genre, classe et prix du ticket). Cependant, le genre est une variable catégorielle qu'il faut transformer en variable numérique :

In [10]:
df["Sex"] = df["Sex"].map({"male": 0, "female": 1}) # remplace male par 0 et female par 1
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,0,22.0,7.2500
1,1,1,1,38.0,71.2833
2,1,3,1,26.0,7.9250
3,1,1,1,35.0,53.1000
4,0,3,0,35.0,8.0500


## Standardisation

Nous remarquons que les attributs sont sur des échelles très différentes (par exemple, l'âge est entre 0 et 80, alors que la classe du billet est entre 1 et 3). Les différences d'âge contriburaient alors beaucoup plus dans les calculs de distance, ce qui ferait que l'âge aurait un poids plus important que la classe du billet pour la prédiction. Pour éviter cela, on va standardiser les données, c'est-à-dire les transformer de manière à ce que chaque attribut ait une moyenne nulle et un écart-type égal à 1.  
Si un attribut $x$ a une moyenne $\bar{x}$ et un écart-type $\sigma$, on peut le standardiser en le remplaçant par :

$$\frac{x - \bar{x}}{\sigma}$$

````{admonition} Question
 Écrire une fonction `standardiser(df, c)` qui standardise la colonne `c` du dataframe `df`. L'utiliser pour standardiser les colonnes `Age`, `Fare`, `Pclass` et `Sex`.
````

In [12]:
for c in ["Age", "Fare", "Pclass", "Sex"]:
    standardiser(df, c)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,0.827377,-0.737695,-0.565736,-0.502445
1,1,-1.566107,1.355574,0.663861,0.786845
2,1,0.827377,1.355574,-0.258337,-0.488854
3,1,-1.566107,1.355574,0.433312,0.420730
4,0,0.827377,-0.737695,0.433312,-0.486337


## Distance

Pour la question suivante, on rappelle comment accéder aux attributs d'une donnée :

In [13]:
p = df.loc[0] # 1er passager
p["Age"], p["Fare"], p["Pclass"], p["Sex"] # attributs de p

(-0.565736461074875,
 -0.5024451714361915,
 0.8273772438659676,
 -0.7376951317802897)

````{admonition} Question
 Écrire une fonction `distance(p1, p2)` qui calcule la distance euclidienne entre les passagers `p1` et `p2`. On prendra en compte tous les attributs sauf `Survived`.
````

In [15]:
distance(df.loc[0], df.loc[1]) # distance entre les deux premiers passagers

3.644820996221396

## Séparation des données

On sépare les données en deux : une partie `train` utilisée pour la prédiction, et une partie `test` utilisée pour évaluer la qualité de la prédiction :

In [16]:
train = df.sample(frac=0.8,random_state=0)
test = df.drop(train.index)
print("nombre de données dans train :", len(train))
print("nombre de données dans test :", len(test))

nombre de données dans train : 713
nombre de données dans test : 178


## Algorithmes des plus proches voisins

````{admonition} Question
 Écrire une fonction `voisins(x, k)` qui renvoie les indices des $k$ plus proches voisins de `x` dans `train`.
````

In [18]:
voisins(test.iloc[0], 5)

[446, 651, 546, 427, 389]

````{admonition} Question
 Écrire une fonction `plus_frequent(L)` qui renvoie l'élément le plus fréquent d'une liste `L`.
````

In [20]:
plus_frequent([2, 1, 5, 1, 2, 5, 5])

5

````{admonition} Question
 Écrire une fonction `knn(x, k)` qui renvoie la prédiction de survie de `x` en utilisant l'algorithme des $k$ plus proches voisins.
````

In [22]:
knn(test.iloc[0], 5)

1

## Analyse des résultats

````{admonition} Question
 Écrire une fonction `precision(k)` qui renvoie la précision de l'algorithme des $k$ plus proches voisins en utilisant `k` voisins.
````

In [24]:
precision(3)

0.797752808988764

````{admonition} Question
 Écrire une fonction `plot_precision(kmax)` qui trace la précision pour $k$ variant de $1$ à `kmax`. Quelle est la meilleure précision obtenue ? Quelle est le nombre de voisins optimal ?
````

## Kaggle

Ce TP provient d'une compétition Kaggle : [Titanic: Machine Learning from Disaster](www.kaggle.com/c/titanic).  
Avec KNN, j'obtiens un score de $\approx 0.763$... Ce qui n'est pas terrible car l'exemple de submission basée uniquement sur le genre donne un score de $\approx 0.765$.

````{admonition} Question
 Soumettre le fichier `submission.csv` obtenu par le code ci-dessus sur [Kaggle](https://www.kaggle.com/competitions/titanic/submissions) et essayer d'obtenir un score plus élevé. On pourra essayer d'implémenter les améliorations suggérées à la [fin du cours](https://cpge-itc.github.io/itc2/4_knn/exemple/knn_iris.html).
````

In [33]:
submission = pd.read_csv('test.csv')
for c in "Age", "Fare":
    submission[c].fillna(submission[c].median(), inplace=True)
submission["Sex"] = submission["Sex"].map({"male": 0, "female": 1})
for c in ["Age", "Fare", "Pclass", "Sex"]:
    standardiser(submission, c)
    
submission["Survived"] = [knn(submission.iloc[i], 5) for i in range(len(submission))]

submission[["PassengerId", "Survived"]].to_csv('submission.csv', index=False)